# Installing reqs

In [ ]:
pip install pandas scikit-learn matplotlib seaborn wordcloud tqdm PyPDF2 gensim smart-open nltk python-dotenv langchain langchain_community openai langchain_openai chromadb langchain_huggingface pypdf 

# Read in files and load the vector storage

In [ ]:
# Needs cleaning:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import openai
from tqdm import tqdm
from PyPDF2 import PdfReader
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud 
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI # error? 
from langchain_openai import ChatOpenAI # sub
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS, DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.document_loaders.pdf import PyPDFLoader, PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain.vectorstores import Chroma
import chromadb
import torch
import re
# Wendy's
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.pdf import PyPDFLoader, PyPDFDirectoryLoader
from langchain_community.callbacks import get_openai_callback
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer
import openai
import os
import chromadb
import torch
import re
from typing import List

MODEL_NAME_KBLAB = 'KBLab/sentence-bert-swedish-cased'
MODEL_NAME_KB = 'KB/bert-base-swedish-cased'
MODEL_NAME_INTFLOAT = 'intfloat/multilingual-e5-large-instruct'
MODEL_NAME_KB_LAMMA3dot1 = 'meta-llama/Meta-Llama-3.1-8B'

PATH_DB = './db'
COLLECTION_NAME = 'policy_collection'
# FILE_PATH = './src/documents/Alkohol- och drogpolicy.pdf'
# DIR_PATH = './src/documents'


# Data Preparation
directory_path_eng = '/Users/kailashdejesushornig/Documents/GitHub/Policydokument/KaisProject/data/all_files_eng/no duplicates'
directory_path_swe = '/Users/kailashdejesushornig/Documents/GitHub/Policydokument/KaisProject/data/all_files_swe/no duplicates'

DIR_PATH = directory_path_eng
doc_names = os.listdir(DIR_PATH)

# change '.PDF' to '.pdf'
for doc_name in doc_names:
    if doc_name.endswith('.PDF'):
        old_file_path = os.path.join(DIR_PATH, doc_name)
        new_file_name = doc_name[:-4] + '.pdf'
        new_file_path = os.path.join(DIR_PATH, new_file_name)
        os.rename(old_file_path, new_file_path)

# load pdf document. Use PyPDFDirectoryLoader for loading files in directory.
loader = PyPDFDirectoryLoader(DIR_PATH)
documents = loader.load()
documents[:1]
print(len(documents))


# Split documents into chunks

In [4]:
def save_to_dir(texts: List[str], file_name: str, dir_path: str ='outputs'):
    """
    Save text output to a .txt file in an `outputs` directory. 
    """
    
    # Create an output directory if it doesn't exist.
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
        print("Folder %s created!" % dir_path)

    # Run this to write the answer to a txt file in the output folder
    file_path = dir_path + '/' +  file_name + '.txt'
    open(file_path, 'w').close()
    for text in texts:
        with open(file_path, 'a', encoding='utf-8') as f:
            f.write(text + "\n\n")

def split_documents(chunk_size, documents, tokenizer_name):
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """

    # We use a hierarchical list of separators specifically tailored for splitting Markdown documents
    # This list is taken from LangChain's MarkdownTextSplitter class
    MARKDOWN_SEPARATORS = [
        "\n\n\n\n",
        "\n\n\n",
        "\n\n",
        "\n",
        ".",
        ",",
        " ",
        "",
    ]
    # Remove all whitespaces between newlines e.g. \n \n \n \n --> \n\n\n\n
    for doc in documents:
        doc.page_content = re.sub('(?<=\\n) (?=\\n)', '', doc.page_content)

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer=AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in documents:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

def upload_data(documents, embedding_model, chunk_size, collection_name, persist_dir):
    """
    Create a Chroma vectorstore from a list of documents.
    """
    
    # Split the documents to chunks
    docs = split_documents(
        chunk_size,  # Choose a chunk size adapted to our model
        documents,
        tokenizer_name=MODEL_NAME_KBLAB,
    )

    # Write chunk texts to txt file
    # chunks = [chunk.page_content for chunk in docs]
    # save_to_dir(chunks, 'chunks')
    
    # Create Chroma DB with document chunks
    print(f"Added {len(docs)} chunks to ChromaDB")
    return Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        collection_name=collection_name,
        persist_directory=persist_dir
    )

# Chroma vectorstore and Embedding model

### Get embedding models maximimum sequence length (not strict)

In [5]:
# from sentence_transformers import SentenceTransformer
# print(f"Model's maximum sequence length: {SentenceTransformer(MODEL_NAME_KBLAB).max_seq_length}")

### Initialize embedding model

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   # Check for CUDA enabled GPU
embedding_model = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_KBLAB, # Provide the pre-trained model's path
    model_kwargs={'device':device}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': True} # Set `True` for cosine similarity
)

### Upload docuements to ChromaDB and create a vectorstore
#### Run this code if database is empty.

In [7]:
# Load pdf document. Use PyPDFDirectoryLoader for loading files in directory.
# loader = PyPDFLoader(FILE_PATH)
# loader = PyPDFDirectoryLoader(DIR_PATH)
# documents = loader.load()
# print('Nr. of documents:', len(documents))
# print('A Document object:', documents[:1])

# Create vectorstore with the documents
# vectorstore = upload_data(documents, embedding_model, 768, COLLECTION_NAME, PATH_DB)

Added 714 chunks to ChromaDB


### Initialize existing persisting storage

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
client = chromadb.PersistentClient(path=PATH_DB)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model,
    client=client
)
vectorstore.get()

# Retrieval and generation: Create an agent2agent dialogue pipeline

### The dummy version with meat-eating debate

In [10]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from datetime import datetime
import time
import openai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# These agents a real chatterboxes, they need some restrains
output_length = 50

# Define system prompts for the two agents
devil_system_prompt = f"""
You are the Devil's Advocate. You will have a debate with the Angel's Advocate. Your mission is to make your case that eating meat is ethically right. 
Always meet your opponent's most recent arguement first and indicate this by writing "Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

angel_system_prompt = f"""
You are the Angel's Advocate. You will have a debate with the Devil's Advocate. Your mission is to make the case that eating meat is ethically wrong. 
Always meet your opponent's most recent arguement first and indicate this by writing "\n Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "\n New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

# Initialize ChatOpenAI instances
devil_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")
angel_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")

# Define initial task prompt for the devil agent
task_prompt = "Discuss how eating meat is ethically right or wrong. You will start by making presenting your point of view on the matter."

# Function to create a debate between the two agents
def run_debate(devil_prompt, angel_prompt, task_prompt, num_rounds=3):
    devil_message = task_prompt
    dialogue = []

    for round_num in range(num_rounds):
        # Devil agent's turn
        devil_response = devil_agent.invoke([{"role": "system", "content": devil_prompt}, {"role": "user", "content": devil_message}])
        devil_message = devil_response.content
        dialogue.append(f"\n \n #####Devil: {devil_message}")
        
        # Angel agent's turn
        angel_response = angel_agent.invoke([{"role": "system", "content": angel_prompt}, {"role": "user", "content": devil_message}])
        angel_message = angel_response.content
        dialogue.append(f"\n \n #####Angel: {angel_message}")
        
        # Prepare for the next round
        devil_message = angel_message

    return dialogue

# Start the timer
start_time = time.time()
# Run the debate
debate_dialogue = run_debate(devil_system_prompt, angel_system_prompt, task_prompt, num_rounds=2)

# Verbose: Print the full dialogue
# for line in debate_dialogue:
    # print(line)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time
# Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"debate_dialogue_{current_time}.txt"
with open(file_name, "w") as file:
    for line in debate_dialogue:
        file.write(line + "\n")
    file.write(f"\nTime taken: {elapsed_time:.2f} seconds\n")

print(f"Debate dialogue saved to {file_name}")
print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. 

Debate dialogue saved to debate_dialogue_20240729_163517.txt
Time taken: 9.55 seconds


## Full version for loopholes [Work in progress]

In [ ]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from datetime import datetime
import time
import openai
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback


# Load environment variables from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# These agents a real chatterboxes, they need some restrains
output_length = 50 # tokens
discussion_theme = 'It is allowed to drink beer in the Chalmers office after 3 pm on Fridays.' # formulate as a thesis
devil_mission= 'Your mission is to exploit loopholes in the university´s governing documents.'
angel_mission= 'Your mission is to prevent the exploitation of loopholes in the university´s governing documents.'


# Define system prompts for the two agents
devil_system_prompt = f"""
You are the Devil's Advocate. You will have a debate with the Angel's Advocate. 
{devil_mission} You'll advocate for the thesis: {discussion_theme}  
You must build your arguments on the information given in the documents you have from the vector storage. 
Find any relevant documents and make a point based on the information in the document(s), don't forget to reference the source by stating a quote from the document.

Always meet your opponent's most recent arguement first and indicate this by writing "Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

angel_system_prompt = f"""
You are the Angel's Advocate. You will have a debate with the Devil's Advocate. 
{angel_mission} You'll advocate for the thesis: {discussion_theme} 
You must build your arguments on the information given in the documents you have from the vector storage. 
Find the relevant document for the matter, make a point based on the information given in it and reference the source by stating a quote from the documents.

Always meet your opponent's most recent arguement first and indicate this by writing "\n Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "\n New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

# Initialize ChatOpenAI instances
devil_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")
angel_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# Define initial task prompt for the devil agent
task_prompt = "Discuss why the policy does allow you to drink beer during working hours. You will start by making presenting your point of view on the matter."

# Function to create a debate between the two agents
def run_debate(devil_prompt, angel_prompt, task_prompt, num_rounds=3):
    devil_message = task_prompt
    dialogue = []

    for round_num in range(num_rounds):

        def format_docs(docs): # what does this do? 
            return "\n\n".join(doc.page_content for doc in docs)
        
        print(f"__________Turn number {round_num}__________")
        
        # Devil agent's turn
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()} # question here becomes the input for debate
            | devil_system_prompt
            | devil_agent
            | StrOutputParser()
        )
        devil_response = rag_chain.invoke(devil_message)
        devil_message = devil_response.content
        dialogue.append(f"\n \n #####Devil: {devil_message}")
        
        # Angel agent's turn
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | angel_system_prompt
            | angel_agent
            | StrOutputParser()
        )
        angel_response = angel_agent.invoke(devil_message)
        angel_message = angel_response.content
        dialogue.append(f"\n \n #####Angel: {angel_message}")
        
        # Prepare for the next round
        devil_message = angel_message

    return dialogue

# Start the timer
start_time = time.time()

# Run the debate
debate_dialogue = run_debate(devil_system_prompt, angel_system_prompt, task_prompt, num_rounds=2)

# Verbose: Print the full dialogue
# for line in debate_dialogue:
    # print(line)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time

# Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"debate_dialogue_{current_time}.txt"
with open(file_name, "w") as file:
    for line in debate_dialogue:
        file.write(line + "\n")
    file.write(f"\nTime taken: {elapsed_time:.2f} seconds\n")

print(f"Debate dialogue saved to {file_name}")
print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. Or be the judge on who won the debate and what the answer should be.  
# For a more sophisticated solution would include a expert panel that could vote on who would be the winner of the debate with different areas/principles to investigate. 